# Merge de dois datasets

Um dataset foi construído em formato csv, apartir do uso da API do site steamspy >>> https://steamspy.com/api.php


Já para o outro dataset, foi adquirido do github em formato json >>> https://github.com/leinstay/steamdb/blob/main/steamdb.json


para enriquecer os dados, obteve-se algumas colunas como: data de lançamento, tags, gênero e categorias.



---



# Importação das Bibliotecas

In [ ]:
import pandas as pd
import numpy as np

# Leitura dos arquivos

In [ ]:
df = pd.read_csv('steamspy_50k_jogos.csv')
df2 = pd.read_json('steamdb.json')

# Padronização dos ID's

In [ ]:
df2['appid'] = df2['sid']

# Lista de colunas a serem excluídas

In [ ]:
colunas = [
    "sid",
    "store_url",
    "store_promo_url",
    "store_uscore",
    "published_meta",
    "published_stsp",
    "published_hltb",
    "published_igdb",
    "image",
    "name",
    "description",
    "full_price",
    "current_price",
    "discount",
    "platforms",
    "developers",
    "publishers",
    "languages",
    "voiceovers",
    "achievements",
    "gfq_url",
    "gfq_difficulty",
    "gfq_difficulty_comment",
    "gfq_rating",
    "gfq_rating_comment",
    "gfq_length",
    "gfq_length_comment",
    "stsp_owners",
    "stsp_mdntime",
    "hltb_url",
    "hltb_single",
    "hltb_complete",
    "meta_url",
    "meta_score",
    "meta_uscore",
    "grnk_score",
    "igdb_url",
    "igdb_single",
    "igdb_complete",
    "igdb_score",
    "igdb_uscore",
    "igdb_popularity"
]

# Exclusão das colunas

In [ ]:
df2 = df2.drop(colunas, axis=1)

# Merge

In [ ]:
df = pd.merge(df, df2, on='appid', how='inner')

# Salvando

In [ ]:
df.to_csv("Teste.csv")



---



# Código de limpeza e construção inicial

# ⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇

# Importando as bibliotecas

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sn

# Lendo CSV

In [ ]:
df = pd.read_csv('Teste.csv')

# Excluindo colunas desnecessárias

In [ ]:
df = df.drop('score_rank', axis=1)
df = df.drop('Unnamed: 0', axis=1)

# Excluindo Duplicatas

In [ ]:
df.drop_duplicates(inplace=True)

# Tratando coluna, de categórica para média

In [ ]:
def converter_owners(intervalo):
    if isinstance(intervalo, str) and '..' in intervalo:
        partes = intervalo.strip('"').replace(',', '').split(' .. ')
        min_val = int(partes[0])
        max_val = int(partes[1])
        return (min_val + max_val) / 2
    return np.nan

df['owners'] = df['owners'].apply(converter_owners)
df['owners'] = df['owners'].fillna(df['owners'].mean())

# Preenchendo valores nulos

In [ ]:
for col in ['name', 'developer', 'publisher']:
    df[col] = df[col].fillna('NO DATA')
df['price'].fillna(df['price'].mean())
df['initialprice'].fillna(df['initialprice'].mean())
df['discount'].fillna(df['discount'].mode()[0])

# Corrigindo preços de centavos de dólares para dólares

In [ ]:
df['price'] = df['price'] / 100
df['initialprice'] = df['initialprice'] / 100

# Criação de novas colunas

In [ ]:

df["total_reviews"] = df["positive"] + df["negative"]
df["avg_playtime_hours"] = df["average_forever"] / 60
df["recent_playtime_hours"] = df["average_2weeks"] / 60

# Arredondamento e conversão de tipo das novas colunas

In [ ]:

df["total_reviews"] = df["total_reviews"].round(2)
df["owners"] = df["owners"].astype(int)
df["avg_playtime_hours"] = df["avg_playtime_hours"].round(2)
df["recent_playtime_hours"] = df["recent_playtime_hours"].round(2)


# Histograma dos preços

In [ ]:

ax = sn.histplot(data=df, x='price', kde=True)
ax.figure.set_size_inches(8, 4)
ax.set_title('Histograma do Valor dos Jogos', fontsize=16)
ax.set_xlabel('Valor (US$)', fontsize=14)
plt.show()

# Boxplot dos preços

In [ ]:
ax = sn.boxplot(x='price', data=df, width=0.4)
ax.figure.set_size_inches(8, 4)
ax.set_title('Boxplot da Distribuição de Preço', fontsize=16)
ax.set_xlabel('Preço (US$)', fontsize=14)
plt.show()

# Salvando em um novo csv

In [ ]:
df.to_csv('games_dataset.csv', index=False)